## Importar librerías

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.base import clone
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import xgboost as xgb
import multiprocessing
from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, wait
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
import warnings
warnings.filterwarnings('ignore')

## Cargar y preparar datos

In [2]:
def create_training_dataset(df, trade_type):
    df = df.drop_duplicates()
    # Filtrar las operaciones del tipo especificado y con profit != 0
    df_trade = df[(df['type'] == trade_type) & (df['profit'] != 0)].copy()
    # Separar en ganadoras y perdedoras
    df_winning = df_trade[df_trade['profit'] > 0]
    df_losing = df_trade[df_trade['profit'] < 0]
    n_winning = len(df_winning)
    n_losing = len(df_losing)
    print(f"Tipo de operación: {'Buy' if trade_type == 1 else 'Sell'}")
    print(f"Total Ganadoras: {n_winning}")
    print(f"Total Perdedoras: {n_losing}")
    # Verificar que hay suficientes datos
    if n_winning == 0 or n_losing == 0:
        print(f"No hay suficientes datos para {'compras' if trade_type == 1 else 'ventas'} para entrenar el modelo.")
        return False
    # Equilibrar las clases
    if n_winning <= n_losing:
        n_samples_per_class = n_winning
        # Seleccionar todas las ganadoras
        selected_winning = df_winning.copy()
        # Ordenar las perdedoras por pérdida de mayor a menor (menor profit a mayor)
        df_losing_sorted = df_losing.sort_values(by='profit', ascending=True)
        # Seleccionar las perdedoras con mayor pérdida
        selected_losing = df_losing_sorted.head(n_samples_per_class)
    else:
        n_samples_per_class = n_losing
        # Seleccionar todas las perdedoras
        selected_losing = df_losing.copy()
        # Ordenar las ganadoras por profit de mayor a menor
        df_winning_sorted = df_winning.sort_values(by='profit', ascending=False)
        # Seleccionar las ganadoras con mayor profit
        selected_winning = df_winning_sorted.head(n_samples_per_class)
    print(f"Se seleccionarán {n_samples_per_class} muestras por clase.")
    # Combinar las muestras seleccionadas
    df_training = pd.concat([selected_winning, selected_losing], ignore_index=True)
    # Añadir la columna 'Target' basada en el profit
    df_training['target'] = df_training['profit'].apply(lambda x: 1 if x > 0 else 0)
    # Seleccionar las columnas necesarias (todas menos las dos últimas para el conjunto principal,
    # y todas las columnas de los subconjuntos excepto la última)
    # Suponiendo que las dos últimas columnas en el conjunto principal son 'type' y 'profit'
    feature_columns = df.columns[:-2]
    df_training = df_training[feature_columns.tolist() + ['target']]
    # Mezclar los datos
    df_training = df_training.sample(frac=1).reset_index(drop=True)
    # Eliminar posibles missings
    if(df_training.isna().values.any()):
        df_training=df_training.dropna()
    # retunr df
    return df_training

In [3]:
# Cargar, limpiar y preparar datasets
def load_dataset(df):
    # Preparación de los datos de compra
    df_buy = create_training_dataset(df, trade_type=1)
    X_buy_train = df_buy.drop(columns='target')
    y_buy_train = df_buy['target']
    # Preparación de los datos de venta
    df_sell = create_training_dataset(df, trade_type=-1)
    X_sell_train = df_sell.drop(columns='target')
    y_sell_train = df_sell['target']
    return X_buy_train.values, y_buy_train.values, X_sell_train.values, y_sell_train.values

In [4]:
file_folder = r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/"
# Leer los archivos CSV
file_pattern = os.path.join(file_folder, 'training_dataset_*.csv')
df_file_path = glob.glob(file_pattern)
df = pd.read_csv(df_file_path[0])
X_buy_train, y_buy_train, X_sell_train, y_sell_train = load_dataset(df)
print(f"Buy  -> Trades: {X_buy_train.shape[0]} Features: {X_buy_train.shape[1]}")
print(f"Sell -> Trades: {X_sell_train.shape[0]} Features: {X_sell_train.shape[1]}")

Tipo de operación: Buy
Total Ganadoras: 3045
Total Perdedoras: 2511
Se seleccionarán 2511 muestras por clase.
Tipo de operación: Sell
Total Ganadoras: 2798
Total Perdedoras: 2182
Se seleccionarán 2182 muestras por clase.
Buy  -> Trades: 5022 Features: 86
Sell -> Trades: 4364 Features: 86


## Entrenar modelos

In [5]:
class GeneticAlgorithmCV:
    def __init__(
        self,
        estimator,
        param_grid,
        cv=None,
        scoring=None,
        pop_size=50,
        generations=15,
        early_stopping_rounds=1,
        crossover_initial=0.1,
        crossover_end=0.9,
        mutation_initial=0.9,
        mutation_end=0.1,
        elitism=True,
        elite_size=5,
        tournament_size=3,
        n_random=10,
        n_jobs=-1,
        verbose=False
    ):
        self.estimator = estimator
        self.param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.pop_size = pop_size
        self.generations = generations
        self.early_stopping_rounds = early_stopping_rounds
        self.crossover_initial = crossover_initial
        self.crossover_end = crossover_end
        self.mutation_initial = mutation_initial
        self.mutation_end = mutation_end
        self.elitism = elitism
        self.elite_size = elite_size
        self.tournament_size = tournament_size
        self.n_random = n_random
        self.n_jobs = n_jobs
        self.verbose = verbose
        self.best_params_ = None
        self.best_score_ = None

    def decode_chromosome(self, chromosome):
        param_values = {}
        for i, key in enumerate(self.param_grid.keys()):
            gene = chromosome[i]
            param_info = self.param_grid[key]
            low = param_info['low']
            high = param_info['high']
            if param_info['type'] == 'int':
                value = int(np.round(gene * (high - low) + low))
            elif param_info['type'] == 'float':
                value = gene * (high - low) + low
            param_values[key] = value
        return param_values

    def initialize_population(self):
        chromosome_length = len(self.param_grid)
        population = np.random.uniform(low=0.0, high=1.0, size=(self.pop_size, chromosome_length))
        return population

    def evaluate_population(self, population, X_train, y_train):
        # Definir la función que evaluará un individuo
        def evaluate_individual(chromosome):
            params = self.decode_chromosome(chromosome)
            scores = []
            for train_idx, val_idx in self.cv.split(X_train, y_train):
                X_tr, X_val = X_train[train_idx], X_train[val_idx]
                y_tr, y_val = y_train[train_idx], y_train[val_idx]
                # Clonar el estimador para evitar efectos colaterales
                model = clone(self.estimator)
                model.set_params(**params)
                model.fit(X_tr, y_tr)
                y_pred = model.predict(X_val)
                # Calcular la puntuación utilizando el scorer
                score = self.scoring(y_val, y_pred)
                scores.append(score)
            fitness = np.mean(np.array(scores))
            return fitness
        # Paralelizar la evaluación de la población
        if self.n_jobs == -1:
            n_jobs = multiprocessing.cpu_count()
        else:
            n_jobs = self.n_jobs
        fitnesses = Parallel(n_jobs=n_jobs)(
            delayed(evaluate_individual)(chromosome) for chromosome in population
        )
        return np.array(fitnesses)

    def select_parents(self, population, fitnesses):
        selected = []
        for _ in range(len(population)):
            indices = np.random.randint(0, len(population), size=self.tournament_size)
            best_idx = indices[np.argmax(fitnesses[indices])]
            selected.append(population[best_idx])
        return np.vstack(selected)

    def crossover(self, parents, crossover_rate):
        offspring = []
        for i in range(0, len(parents), 2):
            parent1 = parents[i].copy()
            parent2 = parents[(i+1) % len(parents)].copy()
            if np.random.rand() < crossover_rate:
                point = np.random.randint(1, len(parent1))
                child1 = np.concatenate((parent1[:point], parent2[point:]))
                child2 = np.concatenate((parent2[:point], parent1[point:]))
                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.append(parent1)
                offspring.append(parent2)
        return np.vstack(offspring)

    def mutate(self, offspring, mutation_rate, mutation_scale=0.1):
        for chromosome in offspring:
            if np.random.rand() < mutation_rate:
                gene_idx = np.random.randint(0, len(chromosome))
                mutation = np.random.normal(0, mutation_scale)
                chromosome[gene_idx] += mutation
                chromosome[gene_idx] = np.clip(chromosome[gene_idx], 0.0, 1.0)
        return offspring

    def generate_random_individuals(self, n_random):
        chromosome_length = len(self.param_grid)
        random_chromosomes = np.empty((n_random, chromosome_length), dtype=np.float32)
        for i, key in enumerate(self.param_grid.keys()):
            grid = self.param_grid[key]
            low = grid['low']
            high = grid['high']
            if grid['type'] == 'int':
                sampled = np.random.randint(low, high + 1, size=n_random)
                normalized = (sampled - low) / (high - low)
                random_chromosomes[:, i] = normalized.astype(np.float32)
            elif grid['type'] == 'float':
                sampled = np.random.uniform(low, high, size=n_random)
                normalized = (sampled - low) / (high - low)
                random_chromosomes[:, i] = normalized.astype(np.float32)
            else:
                raise ValueError(f"Tipo de parámetro no soportado: {grid['type']}")
        return random_chromosomes

    def fit(self, X_train, y_train):
        if self.cv is None:
            self.cv = StratifiedKFold(n_splits=5, shuffle=True)
        chromosome_length = len(self.param_grid)
        population = self.initialize_population()
        best_overall_fitness = -np.inf
        best_overall_chromosome = None
        no_improvement_generations = 0

        for generation in range(self.generations):
            if self.verbose:
                print(f"Generación [{generation+1}]")
            crossover_rate = self.crossover_initial * ((self.crossover_end / self.crossover_initial) ** (generation / self.generations))
            mutation_rate = self.mutation_initial * ((self.mutation_end / self.mutation_initial) ** (generation / self.generations))
            if self.verbose:
                print(f"Crossover Rate: {crossover_rate:.4f}, Mutation Rate: {mutation_rate:.4f}")
            fitnesses = self.evaluate_population(population, X_train, y_train)
            current_best_fitness = np.max(fitnesses)
            if self.verbose:
                print(f"Mejor fitness en generación [{generation+1}]: {current_best_fitness}")
            if current_best_fitness > best_overall_fitness:
                best_overall_fitness = current_best_fitness
                best_idx = np.argmax(fitnesses)
                best_overall_chromosome = population[best_idx]
                no_improvement_generations = 0
            else:
                no_improvement_generations += 1
            if no_improvement_generations >= self.early_stopping_rounds:
                if self.verbose:
                    print(f"No hubo mejora en el fitness por {self.early_stopping_rounds} generaciones consecutivas. Deteniendo el algoritmo.")
                    print(f"El mejor fitness: {best_overall_fitness}")
                break
            if self.elitism:
                sorted_indices = np.argsort(fitnesses)[::-1]
                elites = population[sorted_indices[:self.elite_size]]
            else:
                elites = None
            # Seleccionar padres
            parents = self.select_parents(population, fitnesses)
            # Generar descendencia mediante cruza
            offspring = self.crossover(parents, crossover_rate=crossover_rate)
            # Aplicar mutaciones a la descendencia
            offspring = self.mutate(offspring, mutation_rate=mutation_rate)
            # Inyección de individuos aleatorios
            random_individuals = self.generate_random_individuals(self.n_random)
            offspring = np.vstack((offspring, random_individuals))
            # Mantener el tamaño de la población
            if self.elitism and elites is not None:
                population = np.vstack((elites, offspring))
            else:
                population = offspring
            # Si la población excede el tamaño, seleccionar los mejores
            if len(population) > self.pop_size:
                fitnesses = self.evaluate_population(population, X_train, y_train)
                sorted_indices = np.argsort(fitnesses)[::-1]
                population = population[sorted_indices[:self.pop_size]]
                    
        self.best_params_ = self.decode_chromosome(best_overall_chromosome)
        self.best_score_ = best_overall_fitness.get()  # Convertir a float de Python
        # Entrenar el mejor estimador
        self.best_estimator_ = clone(self.estimator)
        self.best_estimator_.set_params(**self.best_params_)
        self.best_estimator_.fit(X_train, y_train)
        return self

In [6]:
def train_model_buy(X_train, y_train, param_grid):
    try:
        # Definir el pipeline con placeholders (compras)
        estimator = Pipeline([
            ('scaler', RobustScaler()),
            ('selector', GenericUnivariateSelect(score_func=f_classif, mode='percentile')),
            ('reducer', PCA(n_components=0.95)),
            ('classifier', xgb.XGBClassifier(eval_metric='mlogloss', tree_method='gpu_hist', predictor='gpu_predictor', verbosity=0))
        ])
    # Crear una instancia de GeneticAlgorithmCV
        ga_search = GeneticAlgorithmCV(
            estimator=estimator,
            param_grid=param_grid,
            cv=StratifiedKFold(n_splits=5, shuffle=True),
            scoring=accuracy_score,
            pop_size=50,
            generations=15,
            tournament_size=3,
            crossover_initial=0.1,
            crossover_end=0.9,
            mutation_initial=0.9,
            mutation_end=0.1,
            elitism=True,
            elite_size=5,
            n_random=10,
            early_stopping_rounds=1,
            n_jobs=-1,
            verbose=True
        )
        # Entrenar el modelo utilizando el algoritmo genético
        ga_search.fit(X_train, y_train)
    except Exception as e:
        print(f"Error en train_model_buy: {e}")
        raise
    # Obtener los mejores parámetros y el mejor estimador
    best_params = ga_search.best_params_
    print("Mejores parámetros encontrados para compras:", best_params)
    print("Mejor puntuación de validación para compras:", ga_search.best_score_)
    # Retornar mejores parámetros
    return best_params

In [7]:
def train_model_sell(X_train, y_train, param_grid):
    try:
        # Definir el pipeline con placeholders (ventas)
        estimator = Pipeline([
            ('scaler', RobustScaler()),
            ('selector', GenericUnivariateSelect(score_func=f_classif, mode='percentile')),
            ('reducer', PCA(n_components=0.95)),
            ('classifier', xgb.XGBClassifier(eval_metric='mlogloss', tree_method='gpu_hist', predictor='gpu_predictor', verbosity=0))
        ])
        # Crear una instancia de GeneticAlgorithmCV
        ga_search = GeneticAlgorithmCV(
            estimator=estimator,
            param_grid=param_grid,
            cv=StratifiedKFold(n_splits=5, shuffle=True),
            scoring=accuracy_score,
            pop_size=50,
            generations=15,
            tournament_size=3,
            crossover_initial=0.1,
            crossover_end=0.9,
            mutation_initial=0.9,
            mutation_end=0.1,
            elitism=True,
            elite_size=5,
            n_random=10,
            early_stopping_rounds=1,
            n_jobs=-1,
            verbose=True
        )
        # Entrenar el modelo utilizando el algoritmo genético
        ga_search.fit(X_train, y_train)
    except Exception as e:
        print(f"Error en train_model_buy: {e}")
        raise
    # Obtener los mejores parámetros y el mejor estimador
    best_params = ga_search.best_params_
    print("Mejores parámetros encontrados para compras:", best_params)
    print("Mejor puntuación de validación para compras:", ga_search.best_score_)
    # Retornar mejores parámetros
    return best_params

In [8]:
# Definir espacio de hiperparámetros
param_grid = {
    'selector__param': {'type': 'int', 'low': 1, 'high': 100},
    'classifier__n_estimators': {'type': 'int', 'low': 50, 'high': 500},
    'classifier__max_depth': {'type': 'int', 'low': 3, 'high': 10},
    'classifier__learning_rate': {'type': 'float', 'low': 0.01, 'high': 0.3},
    'classifier__subsample': {'type': 'float', 'low': 0.6, 'high': 1.0},
    'classifier__colsample_bytree': {'type': 'float', 'low': 0.6, 'high': 1.0},
    'classifier__gamma': {'type': 'float', 'low': 0.0, 'high': 0.5},
    'classifier__min_child_weight': {'type': 'int', 'low': 1, 'high': 10},
    'classifier__reg_alpha': {'type': 'float', 'low': 0.0, 'high': 1.0},
    'classifier__reg_lambda': {'type': 'float', 'low': 0.0, 'high': 1.0}
}

In [ ]:
# Entrenar modelos simultáneamente
with ThreadPoolExecutor(max_workers=2) as executor:
    # enviar tareas de entrenamiento
    future_buy = executor.submit(train_model_buy, X_buy_train, y_buy_train, param_grid)
    future_sell = executor.submit(train_model_sell, X_sell_train, y_sell_train, param_grid)
    # esperar a que todas las tareas terminen
    futures = [future_buy, future_sell]
    print("Esperando que las tareas finalicen...")
    wait(futures)
    print("¡Todas las tareas han terminado!")
    # Obtener resultados una vez que ambas tareas han terminado
    model_buy = future_buy.result()
    model_sell = future_sell.result()

Generación [1]
Crossover Rate: 0.1000, Mutation Rate: 0.9000
Generación [1]
Crossover Rate: 0.1000, Mutation Rate: 0.9000
Esperando que las tareas finalicen...
Mejor fitness en generación [1]: 0.5279540653866768
Mejor fitness en generación [1]: 0.563921825137262


## Exportar modelos a formato ONNX

In [11]:
def save_onnx_models(mql5_files_folder):
    try:
        update_registered_converter(
            xgb.XGBClassifier,
            "XGBClassifier",
            calculate_linear_classifier_output_shapes,
            convert_xgboost,
            options={'nocl': [True, False], 'zipmap': [True, False, 'columns']}
        )
        model_buy_onnx = convert_sklearn(
            model_buy,
            'pipeline_buy_xgboost',
            [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        model_sell_onnx = convert_sklearn(
            model_sell,
            'pipeline_sell_xgboost',
            [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
            f.write(model_buy_onnx.SerializeToString())
        with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
            f.write(model_sell_onnx.SerializeToString())
    except Exception as e:
        print(f"Error en exportar los modelos: {e}")
        raise
    print("Modelos ONNX exportados correctamente")

In [13]:
save_onnx_models(r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files')

Modelos ONNX exportados correctamente
